# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 10 2022 7:43AM,239300,16,ACG-2102490292,ACG North America LLC,Released
1,Jun 10 2022 7:33AM,239298,19,DEX0006984,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
2,Jun 10 2022 7:33AM,239298,19,DEX0006985,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
3,Jun 9 2022 5:16PM,239294,21,399280656,"NBTY Global, Inc.",Released
4,Jun 9 2022 4:52PM,239292,21,399290126,"NBTY Global, Inc.",Released
5,Jun 9 2022 4:34PM,239291,21,399290074,"NBTY Global, Inc.",Released
6,Jun 9 2022 4:16PM,239290,19,ACG-2102490295,ACG North America LLC,Released
7,Jun 9 2022 4:02PM,239286,21,399280394,"NBTY Global, Inc.",Released
8,Jun 9 2022 1:39PM,239273,20,8121995,"Exact-Rx, Inc.",Released
9,Jun 9 2022 1:33PM,239271,20,ALU0007028,Alumier Labs Inc.,Completed


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,239291,Released,1
17,239292,Released,1
18,239294,Released,1
19,239298,Released,2
20,239300,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239291,NaN,NaN,1.0
239292,NaN,NaN,1.0
239294,NaN,NaN,1.0
239298,NaN,NaN,2.0
239300,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239218,0.0,0.0,1.0
239220,0.0,0.0,2.0
239223,0.0,0.0,1.0
239253,0.0,0.0,1.0
239258,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239218,0,0,1
239220,0,0,2
239223,0,0,1
239253,0,0,1
239258,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239218,0,0,1
1,239220,0,0,2
2,239223,0,0,1
3,239253,0,0,1
4,239258,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239218,,,1
1,239220,,,2
2,239223,,,1
3,239253,,,1
4,239258,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 10 2022 7:43AM,239300,16,ACG North America LLC
1,Jun 10 2022 7:33AM,239298,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
3,Jun 9 2022 5:16PM,239294,21,"NBTY Global, Inc."
4,Jun 9 2022 4:52PM,239292,21,"NBTY Global, Inc."
5,Jun 9 2022 4:34PM,239291,21,"NBTY Global, Inc."
6,Jun 9 2022 4:16PM,239290,19,ACG North America LLC
7,Jun 9 2022 4:02PM,239286,21,"NBTY Global, Inc."
8,Jun 9 2022 1:39PM,239273,20,"Exact-Rx, Inc."
9,Jun 9 2022 1:33PM,239271,20,Alumier Labs Inc.
37,Jun 9 2022 1:04PM,239262,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 10 2022 7:43AM,239300,16,ACG North America LLC,,,1
1,Jun 10 2022 7:33AM,239298,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,2
2,Jun 9 2022 5:16PM,239294,21,"NBTY Global, Inc.",,,1
3,Jun 9 2022 4:52PM,239292,21,"NBTY Global, Inc.",,,1
4,Jun 9 2022 4:34PM,239291,21,"NBTY Global, Inc.",,,1
5,Jun 9 2022 4:16PM,239290,19,ACG North America LLC,,,1
6,Jun 9 2022 4:02PM,239286,21,"NBTY Global, Inc.",,,1
7,Jun 9 2022 1:39PM,239273,20,"Exact-Rx, Inc.",,,1
8,Jun 9 2022 1:33PM,239271,20,Alumier Labs Inc.,1,3,24
9,Jun 9 2022 1:04PM,239262,10,ISDIN Corporation,6,1,4


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 7:43AM,239300,16,ACG North America LLC,1,,
1,Jun 10 2022 7:33AM,239298,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
2,Jun 9 2022 5:16PM,239294,21,"NBTY Global, Inc.",1,,
3,Jun 9 2022 4:52PM,239292,21,"NBTY Global, Inc.",1,,
4,Jun 9 2022 4:34PM,239291,21,"NBTY Global, Inc.",1,,
5,Jun 9 2022 4:16PM,239290,19,ACG North America LLC,1,,
6,Jun 9 2022 4:02PM,239286,21,"NBTY Global, Inc.",1,,
7,Jun 9 2022 1:39PM,239273,20,"Exact-Rx, Inc.",1,,
8,Jun 9 2022 1:33PM,239271,20,Alumier Labs Inc.,24,3,1
9,Jun 9 2022 1:04PM,239262,10,ISDIN Corporation,4,1,6


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 7:43AM,239300,16,ACG North America LLC,1,,
1,Jun 10 2022 7:33AM,239298,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
2,Jun 9 2022 5:16PM,239294,21,"NBTY Global, Inc.",1,,
3,Jun 9 2022 4:52PM,239292,21,"NBTY Global, Inc.",1,,
4,Jun 9 2022 4:34PM,239291,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 7:43AM,239300,16,ACG North America LLC,1,NaN,NaN
1,Jun 10 2022 7:33AM,239298,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,NaN,NaN
2,Jun 9 2022 5:16PM,239294,21,"NBTY Global, Inc.",1,NaN,NaN
3,Jun 9 2022 4:52PM,239292,21,"NBTY Global, Inc.",1,NaN,NaN
4,Jun 9 2022 4:34PM,239291,21,"NBTY Global, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 7:43AM,239300,16,ACG North America LLC,1,0.0,0.0
1,Jun 10 2022 7:33AM,239298,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,0.0,0.0
2,Jun 9 2022 5:16PM,239294,21,"NBTY Global, Inc.",1,0.0,0.0
3,Jun 9 2022 4:52PM,239292,21,"NBTY Global, Inc.",1,0.0,0.0
4,Jun 9 2022 4:34PM,239291,21,"NBTY Global, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,239262,4,1.0,6.0
12,239261,1,1.0,0.0
16,478558,2,0.0,0.0
18,478471,2,0.0,0.0
19,478588,3,0.0,0.0
20,956987,28,3.0,1.0
21,957163,4,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,239262,4,1.0,6.0
1,12,239261,1,1.0,0.0
2,16,478558,2,0.0,0.0
3,18,478471,2,0.0,0.0
4,19,478588,3,0.0,0.0
5,20,956987,28,3.0,1.0
6,21,957163,4,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,4,1.0,6.0
1,12,1,1.0,0.0
2,16,2,0.0,0.0
3,18,2,0.0,0.0
4,19,3,0.0,0.0
5,20,28,3.0,1.0
6,21,4,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,4.0
1,12,Released,1.0
2,16,Released,2.0
3,18,Released,2.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Completed,6.0,0.0,0.0,0.0,0.0,1.0,0.0
Executing,1.0,1.0,0.0,0.0,0.0,3.0,0.0
Released,4.0,1.0,2.0,2.0,3.0,28.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,1.0,1.0,0.0,0.0,0.0,3.0,0.0
2,Released,4.0,1.0,2.0,2.0,3.0,28.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Completed,6.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Executing,1.0,1.0,0.0,0.0,0.0,3.0,0.0
2,Released,4.0,1.0,2.0,2.0,3.0,28.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()